# Sentinel 1 Mosaic Creation

In [ ]:
'''
## ############################## 
##
## Script Name: S1 Mosaic Creation
##
## Description: Generates Sentinel 1 backscatter mosaics based on user provided dates.
##
## Author: Max Fancourt, Natural England
## 
## Licence: MIT Licence  
## Date Created: 2024-07-17
## 
## Date Last Modified: 2024-07-17
## 
## Versioning: v1.0
## Python Version: 3.11.5
## Dependencies:
## 
## ee       v.0.1.358
## folium   v.0.14.0
##
## ############################## ##
'''

In [5]:
# Import libraries
import ee
import folium
from folium import plugins

# Authenticate connection with Earth Engine, if exception then attempt authentication and then reinitialise connection
try:
    ee.Initialize()
except:
    # Trigger the authentication flow.
    ee.Authenticate()
    ee.Initialize()

In [17]:
# Global variables
EXPORT_FOLDER = "s1_mosaics\summer"
CRS = 'EPSG:27700' # Projection to export data in
ZONE = 14 # Zone to create mosaic for
START_DATE = '2023-06-01' # Start date for mosaicking
END_DATE = '2023-08-08' # End date for mosaicking
TRACKS = [1, 125] # Tracks to restrict to 
# Any specific dates to exclude based on rainfall 
DATES_TO_EXLUDE = [ee.Date.fromYMD(2023,7,26).millis(), ee.Date.fromYMD(2023,7,14).millis(), ee.Date.fromYMD(2023,7,2).millis(), ee.Date.fromYMD(2023,6,20).millis(), ee.Date.fromYMD(2023,6,8).millis(), ee.Date.fromYMD(2023,7,23).millis(), ee.Date.fromYMD(2023,6,5).millis()]

# Spring dates
#START_DATE = '2023-03-01'
#END_DATE = '2023-05-30'

#Autumn dates
#START_DATE = '2022-08-01'
#END_DATE = '2022-10-30'

# Summer dates
#START_DATE = '2023-06-01'
#END_DATE = '2023-07-31'

# Import required assets
TABLE = ee.FeatureCollection("users/MaxFancourt/all_zones_buffered")
AOI = ee.Feature(TABLE.filter(ee.Filter.eq('Zone', ZONE)).first().geometry())

# Visualisation parameteres
S1GRDDISPLAY = {'min':-35, 'max':15, 'bands':["VV", "VH", "VV"]}
S1ARDDISPLAY = {'min':-30, 'max':10, 'bands':["b2", "b1", "b2"]}

# Folium basemaps
# Add custom basemaps to folium
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )
    
}

In [18]:
# Functions
# Function to round timestamp of an image to the day it was taken on  
def RoundDateToNearestDay(image):
    date = ee.Date(image.date())
    return image.set('simpleTime', ee.Date.fromYMD(date.get('year'), date.get('month'), date.get('day')).millis())

# Define a method for displaying Earth Engine image tiles on a folium map
def add_ee_layer(self, ee_object, vis_params, name):
    try:    
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
    
    except:
        print("Could not display {}".format(name))
    
# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [12]:
# Create descending image
# Filter collection 
image1_D = (ee.ImageCollection("COPERNICUS/S1_GRD")
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) # VV polarisation
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')) # VH polarisation
            .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')) # Descending images
            .filter(ee.Filter.eq('instrumentMode', 'IW')) # Interferometric Wide swath mode
            .filter(ee.Filter.inList('relativeOrbitNumber_start', TRACKS)) # Filter to user specified tracks
            .filterDate(START_DATE, END_DATE) # Filter to user specified dates 
            .filterBounds(AOI.geometry()) # Filter to AOI
            .select('VV','VH')
            )

# Round dates to nearest day, exclude images identified as having high rainfall in global variables, create median mosaic of remaining images, and clip to AOI
image1_D_rounded = image1_D.map(RoundDateToNearestDay)
image1_D_final = ee.ImageCollection(image1_D_rounded.filter(ee.Filter.inList('simpleTime', DATES_TO_EXLUDE).Not())).median().clip(AOI.geometry())

# Create ascending image
image1_A = (ee.ImageCollection("COPERNICUS/S1_GRD")
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) # VV polarisation
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')) # VH polarisation
            .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING')) # Ascending images
            .filter(ee.Filter.eq('instrumentMode', 'IW')) # Interferometric Wide swath mode
            .filter(ee.Filter.inList('relativeOrbitNumber_start', TRACKS)) # Filter to user specified tracks
            .filterDate(START_DATE, END_DATE) # Filter to user specified dates 
            .filterBounds(AOI.geometry()) # Filter to AOI
            .select('VV','VH')
            )


# Round dates to nearest day, exclude images identified as having high rainfall in global variables, create median mosaic of remaining images, and clip to AOI
image1_A_rounded = image1_A.map(RoundDateToNearestDay)
image1_A_final = ee.ImageCollection(image1_A_rounded.filter(ee.Filter.inList('simpleTime', DATES_TO_EXLUDE).Not())).median().clip(AOI.geometry())

# Export Ascending and descending images in BNG EPSG27700, at 10m resolution, to attached google drive as geotiffs, depending on size of AOI chosen, output may be split into tiles
task = ee.batch.Export.image.toDrive(image1_D_final,
                                    description = f"zone_{ZONE}_s1_Desc",
                                    folder = EXPORT_FOLDER,
                                    region = AOI.geometry(),
                                    scale = 10,
                                    crs = CRS,
                                    maxPixels = 1e10)
task.start()

task = ee.batch.Export.image.toDrive(image1_A_final,
                                    description = f"zone_{ZONE}_s1_Asc",
                                    folder = EXPORT_FOLDER,
                                    region = AOI.geometry(),
                                    scale = 10,
                                    crs = CRS,
                                    maxPixels = 1e10)
task.start()

Number of images 6
{'type': 'ImageCollection', 'bands': [], 'version': 1694500042016796, 'id': 'COPERNICUS/S1_GRD', 'features': [{'type': 'Image', 'bands': [{'id': 'VV', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [29661, 22294], 'crs': 'EPSG:32630', 'crs_transform': [10, 0, 132823.44007006357, 0, -10, 5616969.452966681]}, {'id': 'VH', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [29661, 22294], 'crs': 'EPSG:32630', 'crs_transform': [10, 0, 132823.44007006357, 0, -10, 5616969.452966681]}], 'version': 1694500042016796, 'id': 'COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20230608T181404_20230608T181429_048898_05E159_DF31', 'properties': {'simpleTime': 1686182400000, 'SNAP_Graph_Processing_Framework_GPF_vers': '8.0.3', 'SLC_Processing_facility_org': 'ESA', 'SLC_Processing_facility_country': 'Italy', 'GRD_Post_Processing_facility_org': 'ESA', 'transmitterReceiverPolarisation': ['VV', 'VH'], 'GRD_Post_Processing_start': 1686249992653, 'sliceNum

In [9]:
# Add the created mosiacis to a folium map for manual visuationation
# Create folium base map
f = folium.Figure(width=2000, height=1000)
my_map = folium.Map(zoom_start=12).add_to(f)

# Add google baselayer
basemaps['Google Maps'].add_to(my_map)

# Add layers
my_map.add_ee_layer(AOI.geometry(), {},"ZONE_BOUNDRY") # Add AOI
my_map.add_ee_layer(image1_D, S1GRDDISPLAY,"image1_D") # Add descending median mosaic
my_map.add_ee_layer(image1_A, S1GRDDISPLAY,"image1_A") # Add ascending median mosiac

# Add a layer control panel to the map
my_map.add_child(folium.LayerControl())

display(my_map)

